In [14]:
#############################
##### the hanabi game #######
#############################

### class: player, discard, played, leftover
### action/decision: (1) give info (2) play card (3) discard

from scipy import *
seed(10027)
cards_ordered = array([[num, color] for num in [1,1,1,2,2,3,3,4,4,5] for color in range(1,6)])
order = range(50)
shuffle(order)
cards_init = cards_ordered [order]
#print cards_init

In [58]:
class deck:
    def __init__(self, cards=zeros(1)):
        if cards.any():
            self._data = cards
            self._knowledge = zeros(shape=cards.shape)
        else:
            self._data = array([])
            self._knowledge = array([])
    def add (self, icard):
        self._data = append(self._data, icard)
    def rm (self, idx):
        self._data = delete(self._data, idx, axis=0)
    def showcard (self):
        return self._data.reshape(-1,2)
    def updateknowledge (self, idx, pos=0): ## pos 0 for num, pos 1 for color
        self._knowledge[idx, pos] = 1
    def showknowledge (self):
        return self._knowledge
    def match (self, icard):
        return where((self._data.T[0] == icard[0]) & (self._data.T[1] == icard[1]))[0]
    def ncard (self):
        return len(self._data)
    
#jia = player() 
#jia = player(cards_init[:4])
#print jia.showcard()
#print jia.showknowledge()

In [59]:
###################################
####### initialize the game #######
###################################

N = 4 # Number of players, each with N number of cards
leftover = deck(cards_init)

#print leftover.showcard()
players = []
for iN in range(N): ## initiate N class objects
    newcard = leftover.showcard()[:N]
    players.append (deck(newcard))
    leftover.rm(range(N))
#    print leftover.ncard(), iN, players[iN].showcard()

In [69]:
def reaction (played, discard, newcard, fuse):
    '''for a new card, decide its destination play or discard'''
    idx_match = played.match(newcard) ## find idx of the newcard in played pile
    if sum(played.showknowledge()[idx_match]): ### discard if newcard already in played
        discard.add(newcard)
        fuse -= 1 ## penalty due to wrong card
    else:
        if newcard[0]==1: ## play, if this is a #1 card
            played.updateknowledge(idx_match,pos=0)
        else: ## check if the card before is played
            idx_before = played.match(newcard-array([1,0]))
            if played.showknowledge()[idx_before][0]:# play, the card before is played
                played.updateknowledge(idx_match,pos=0)
            else: ## discard if the card before is not played
                discard.add(newcard)
                fuse -= 1
                                      
discard = deck()           
played = deck(array([[num, color] for num in range(1,6) for color in range(1,6)]))
fuse = 4
info = 8

######### test: pass #######
#print played.showknowledge()
#inewcard=played.showcard()[0]
#reaction(played,discard,inewcard)
#print 'after reaction'
#print 'discard',discard.showcard()
#print played.showknowledge()

#reaction(played,discard,inewcard)
#print 'after reaction 2'
#print 'discard 2',discard.showcard()
#print played.showknowledge()

In [ ]:
###########################
### make decisions ########
###########################
def decision (played, players, N_current, discard, info):
    #### first, when there is no info point, discard
    
    #### if there are info, decide to play card or give info 

In [ ]:
###########################
### game starts ###########
###########################

discard = deck()           
played = deck(array([[num, color] for num in range(1,6) for color in range(1,6)]))
fuse = 4
info = 8

#while leftover.ncard() > 0 and fuse >0: